### Altair Case Study - Understanding customer segments by their propensity to purchase in campaigns and by their value
### Last Modified Date : 22-Jul-2019
### By : Naveen Rathani

##### Information provided :
1. The attached dataset represents the purchase history of individuals following a marketing campaign (DV = ‘Yes’ is a successful purchase, ‘No’ is an individual who received an offer but did not buy a product)

##### Objectives:
1. Identify the types of individuals\segments likely to buy a product
2. To describe the value of the individual\segments in this sample. Value is defined as (revenue – loss)

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import os

import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string


In [24]:
os.chdir("C:\\Users\\naveen.rathani\\Downloads\\Antuit 2018-2019\\altair")

In [25]:
df = pd.read_csv("purchase_history.csv")
df.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,gender,revenue,loss,hours-per-week,DV
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Male,0,0,40,No
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,Male,0,0,50,No
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,Male,0,0,40,Yes
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Male,7688,0,40,Yes
4,18,?,Some-college,10,Never-married,Unknown,Own-child,Female,0,0,30,No


In [26]:
df['value'] = df.revenue - df.loss
df.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,gender,revenue,loss,hours-per-week,DV,value
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Male,0,0,40,No,0
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,Male,0,0,50,No,0
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,Male,0,0,40,Yes,0
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Male,7688,0,40,Yes,7688
4,18,?,Some-college,10,Never-married,Unknown,Own-child,Female,0,0,30,No,0


# Understanding the data

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16281 entries, 0 to 16280
Data columns (total 13 columns):
age               16281 non-null int64
workclass         16281 non-null object
education         16281 non-null object
education-num     16281 non-null int64
marital-status    16281 non-null object
occupation        16281 non-null object
relationship      16281 non-null object
gender            16281 non-null object
revenue           16281 non-null int64
loss              16281 non-null int64
hours-per-week    16281 non-null int64
DV                16281 non-null object
value             16281 non-null int64
dtypes: int64(6), object(7)
memory usage: 1.6+ MB


In [28]:
# Converting the output variable (purchase outcome or DV) to numeric
df['outcome'] = df['DV'].apply(lambda x : 1 if x == 'Yes' else 0)  # Convert to numeric
df = df.drop('DV',axis=1)
df.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,gender,revenue,loss,hours-per-week,value,outcome
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Male,0,0,40,0,0
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,Male,0,0,50,0,0
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,Male,0,0,40,0,1
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Male,7688,0,40,7688,1
4,18,?,Some-college,10,Never-married,Unknown,Own-child,Female,0,0,30,0,0


In [29]:
for i in np.arange(len(df.loc[:,['workclass','education','education-num',
                                 'marital-status','occupation','relationship','gender']].columns)):
    print(df.loc[:,['workclass','education','education-num',
                                 'marital-status','occupation','relationship','gender']].columns[i])
    print(df.loc[:,['workclass','education','education-num',
                                 'marital-status','occupation','relationship','gender']].iloc[:,i].value_counts(),"\n")

workclass
Private             11210
Self-emp-not-inc     1321
Local-gov            1043
?                     963
State-gov             683
Self-emp-inc          579
Federal-gov           472
Without-pay             7
Never-worked            3
Name: workclass, dtype: int64 

education
HS-grad         5283
Some-college    3587
Bachelors       2670
Masters          934
Assoc-voc        679
11th             637
Assoc-acdm       534
10th             456
7th-8th          309
Prof-school      258
9th              242
12th             224
Doctorate        181
5th-6th          176
1st-4th           79
Preschool         32
Name: education, dtype: int64 

education-num
9     5283
10    3586
13    2670
14     934
11     679
7      638
12     534
6      456
4      309
15     258
5      242
8      224
16     181
3      176
2       79
1       32
Name: education-num, dtype: int64 

marital-status
Married-civ-spouse       7403
Never-married            5434
Divorced                 2190
Widowed        

##### Workclass and occupation have a few missing values (963 ? for workclass & 966 Unknown for occupation)

##### No NAs / weird values found in any of the other columns

In [30]:
# Updating workclass where value = '?' to 'missing'
df['workclass'] = df['workclass'].apply(lambda x : 'missing' if x == '?' else x)  # Convert ? to missing
df['workclass'].value_counts()

Private             11210
Self-emp-not-inc     1321
Local-gov            1043
missing               963
State-gov             683
Self-emp-inc          579
Federal-gov           472
Without-pay             7
Never-worked            3
Name: workclass, dtype: int64

In [52]:
crosstab_temp.columns

Index(['workclass', 0, 1], dtype='object', name='outcome')

In [80]:
# defining segmentation variables
segvars_factors = ['age', 'workclass', 'education', 'education-num', 'marital-status',
       'occupation', 'relationship', 'gender']
segvars_nums = ['hours-per-week', 'value','revenue','loss']

# Checking event rate against each variable
crosstab_full = pd.DataFrame()
for i in segvars_factors:
    crosstab_temp = pd.crosstab(df[i],df['outcome']).reset_index()
    crosstab_temp.columns = ['predictor_levels','outcome_no','outcome_yes']
    crosstab_temp['predictor'] = i
    crosstab_temp['event_rate'] = round(crosstab_temp['outcome_yes']/(crosstab_temp['outcome_yes']+crosstab_temp['outcome_no']),2)
    crosstab_temp.sort_values(by='event_rate', ascending=False, inplace=True)
    crosstab_full = crosstab_full.append(crosstab_temp, ignore_index = True)
print(crosstab_full.head())

  predictor_levels  outcome_no  outcome_yes predictor  event_rate
0               85           1            1       age        0.50
1               90           7            5       age        0.42
2               50         152          112       age        0.42
3               43         196          138       age        0.41
4               46         213          147       age        0.41


In [87]:
pd.pivot_table(data = df, values = 'value', columns = 'outcome', aggfunc = np.mean).reset_index()


outcome,index,0,1
0,value,87.389224,3925.305252


In [89]:
# defining segmentation variables
segvars_factors = ['age', 'workclass', 'education', 'education-num', 'marital-status',
       'occupation', 'relationship', 'gender']
segvars_nums = ['age','hours-per-week', 'value','revenue','loss']

# Checking average value of numeric variable against target variable
crosstab_full_avg = pd.DataFrame()
for i in segvars_nums:
    crosstab_temp = pd.pivot_table(data = df, values = i, columns = 'outcome', aggfunc = np.mean).reset_index()
    crosstab_temp.columns = ['predictor','outcome_no','outcome_yes']
    crosstab_temp['mean_diff'] = round((crosstab_temp['outcome_yes'] - crosstab_temp['outcome_no']),2)
    crosstab_full_avg = crosstab_full_avg.append(crosstab_temp, ignore_index = True)
print(crosstab_full_avg.head())

        predictor  outcome_no  outcome_yes  mean_diff
0             age   37.048010    44.326833       7.28
1  hours-per-week   38.839727    45.411856       6.57
2           value   87.389224  3925.305252    3837.92
3         revenue  143.547004  4115.832033    3972.29
4            loss   56.157780   190.526781     134.37


In [97]:
# Calculating overall event rate
print(df.outcome.value_counts())
print(round(df.outcome.value_counts()*100/df.outcome.value_counts().sum(),2))

0    12435
1     3846
Name: outcome, dtype: int64
0    76.38
1    23.62
Name: outcome, dtype: float64


### Prelim Conclusions:
1. Event rate or purchase rate on the population - 23.6%
2. Avg age of successful purchasers is 44.3 Vs. non-purchases is 37.04 (a difference of 7.3)
3. Avg. hours per week of successful purchasers is also higher (45.4 Vs. 38.8)
4. Avg. revenue generated by successful purchasers is 28 times more than non-successful purchasers (4115.8 Vs. 143.5)
5. EVen further, overall value generated by successful purchasers is 45 times more than non-successful purchasers (3925.3 Vs. 87.4)
6. Contrary to above, Avg. losses from successful purchasers are only 3.5 times more than non-purchasers (190.5 Vs. 56.2)
7. Males have 3 times the event rate of females (30% Vs. 10%). Males also account for 2/3rd of total population & 85% of total events
8. Husband and Wife have almost equal event rate of ~45%  and account for 45% of the total population. Not-in-family, Own-child and Unmarried have extremely low event rates (9.8%, 1.8%, 5.4% respectively)
9. Exec-managerial, Prof-specialty have high event rates (46.5% and 45.5% respectively) ; Protective-serv, Sales, Tech-support have above average (23.6%) event rates (29%, 26.5%, 26.4% respectively)
10. Married-civ-spouse have a high event rate (44.5%) ; never married and divorced have the least event rates (4.5% & 9.5% respectively) but account for 40% of the population
11. Higher education is correlated with higher event rate (bacherlors, masters, doctorates, professional school). Assoc-acdm and Assoc-voc have higher than average event rate
12. self employed inc and federal goverment jobs are correlated with high event rates (54.6% and 40.3% respectively). Private employees make up 69% of total population and have a lower than average event rate @ 21.6%
13. No age value below 32 has a higher than average event rate of 23.6%    



## Assessing Variable power in diffrentiating event from non-event

In [98]:
# Binning numerical variables and creating functions to calculate WoE and IV
max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [99]:
final_iv, IV = data_vars(df,df.outcome)

C:\Users\naveen.rathani\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:85: RuntimeWarning: divide by zero encountered in log
C:\Users\naveen.rathani\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in log


In [100]:
final_iv

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,age,17,31,5658,403,0.071227,5255,0.928773,0.104784,0.422598,-1.394517,0.584461
1,age,32,44,5344,1673,0.313061,3671,0.686939,0.434997,0.295215,0.387636,0.584461
2,age,45,90,5279,1770,0.335291,3509,0.664709,0.460218,0.282187,0.489130,0.584461
3,workclass,Federal-gov,Federal-gov,472,190,0.402542,282,0.597458,0.049402,0.022678,0.778598,0.190664
4,workclass,Local-gov,Local-gov,1043,310,0.297220,733,0.702780,0.080603,0.058947,0.312908,0.190664
5,workclass,Never-worked,Never-worked,3,0,0.000000,3,1.000000,0.000000,0.000241,0.000000,0.190664
6,workclass,Private,Private,11210,2424,0.216236,8786,0.783764,0.630265,0.706554,-0.114259,0.190664
7,workclass,Self-emp-inc,Self-emp-inc,579,316,0.545769,263,0.454231,0.082163,0.021150,1.357070,0.190664
8,workclass,Self-emp-not-inc,Self-emp-not-inc,1321,353,0.267222,968,0.732778,0.091784,0.077845,0.164717,0.190664
9,workclass,State-gov,State-gov,683,177,0.259151,506,0.740849,0.046022,0.040692,0.123094,0.190664


In [102]:
final_iv[['VAR_NAME', 'IV']].drop_duplicates().sort_values(by="IV",ascending = False)

,VAR_NAME,IV
54,relationship,1.520835
32,marital-status,1.370332
39,occupation,0.762323
12,education,0.721643
0,age,0.584461
28,education-num,0.569255
62,revenue,0.313948
68,value,0.313948
66,hours-per-week,0.299333
60,gender,0.294928


#### IV and WoE suggest a stronger ((almost suspicious) than normal IV of relationship, marital status, occupation, education, age
#### Also strong predictors = hours_per_week, gender, workclass

In [103]:
transform_vars_list = df.columns.difference(['outcome'])
transform_prefix = 'new_' # leave this value blank if you need replace the original column values

In [104]:
transform_vars_list

Index(['age', 'education', 'education-num', 'gender', 'hours-per-week', 'loss',
       'marital-status', 'occupation', 'relationship', 'revenue', 'value',
       'workclass'],
      dtype='object')

In [105]:
for var in transform_vars_list:
    small_df = final_iv[final_iv['VAR_NAME'] == var]
    transform_dict = dict(zip(small_df.MAX_VALUE,small_df.WOE))
    replace_cmd = ''
    replace_cmd1 = ''
    for i in sorted(transform_dict.items()):
        replace_cmd = replace_cmd + str(i[1]) + str(' if x <= ') + str(i[0]) + ' else '
        replace_cmd1 = replace_cmd1 + str(i[1]) + str(' if x == "') + str(i[0]) + '" else '
    replace_cmd = replace_cmd + '0'
    replace_cmd1 = replace_cmd1 + '0'
    if replace_cmd != '0':
        try:
            df[transform_prefix + var] = df[var].apply(lambda x: eval(replace_cmd))
        except:
            df[transform_prefix + var] = df[var].apply(lambda x: eval(replace_cmd1))

In [108]:
df['new_age'].value_counts()

-1.394517    5658
 0.387636    5344
 0.489130    5279
Name: new_age, dtype: int64

In [111]:
small_df = final_iv[final_iv['VAR_NAME'] == 'age']

In [112]:
small_df

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,age,17,31,5658,403,0.071227,5255,0.928773,0.104784,0.422598,-1.394517,0.584461
1,age,32,44,5344,1673,0.313061,3671,0.686939,0.434997,0.295215,0.387636,0.584461
2,age,45,90,5279,1770,0.335291,3509,0.664709,0.460218,0.282187,0.489130,0.584461


## Exploring conditional trees and CHAID to identify high 'potential to purchase' paths and hence segments

##### Priority variables: 
1. relationship
2. marital-status (highly correlated with relationship)
3. occupation
4. education
5. age
6. hours per week
7. gender
8. workclass

In [150]:
# Creating categorical variable for age:
df['new_revenue'] = df['revenue'].apply(lambda x : ' eq 0' if x<=0 else 'gt 0')

# Creating categorical variable for age:
df['new_value'] = df['value'].apply(lambda x : 'leq 0' if x<=0 else 'gt 0')

df.to_csv("finaldat.csv")

In [146]:
# Creating categorical variable for age:
df['new_age'] = df['age'].apply(lambda x : '<32' if x<32 else '32-44' if  (x<45) and (x>=32) else '>45')

# Creating categorical variable for age:
df['new_hours-per-week'] = df['hours-per-week'].apply(lambda x : '<41' if x<41 else '>41')

# Creating categorical variable for age:
df['new_revenue'] = df['revenue'].apply(lambda x : '=0' if x<=0 else '>0')

# Creating categorical variable for age:
df['new_value'] = df['value'].apply(lambda x : '=0' if x<=0 else '>0')

modellingvars = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 
                 'gender','new_age','new_hours-per-week']

## decision tree
from sklearn import tree
dtree = tree.DecisionTreeClassifier(criterion='gini') # for classification, here you can change the algorithm as gini or entropy (information gain) by default it is gini  
# model = tree.DecisionTreeRegressor() for regression
# Train the model using the training sets and check score
one_hot_data = pd.get_dummies(df[modellingvars])
one_hot_data

dtree.fit(one_hot_data, df['outcome'])
dtree.score(one_hot_data, df['outcome'])
#Predict Output
# predicted= model.predict(x_test)

from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

# Specifing path for dot file.
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/graphviz-2.38/release/bin/'
dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [145]:
# plotting tree with max_depth=3
# Specifing path for dot file.
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/graphviz-2.38/release/bin/'
dot_data = StringIO()  
export_graphviz(dtree, out_file=dot_data,
                feature_names=one_hot_data.columns, filled=True,rounded=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("dt_purchase_chance.pdf")

True